### Definiendo modelos con fórmulas

Definir un modelo mediante una matriz de diseño con las variables explicativas no es especialmente complicado. Sobre todo en Statsmodels, que admite que usemos las columnas de un DataFrame de Pandas directamente.

Sin embargo, trabajar con matrices de modelo cuando quieres probar distintas variantes (quitando o añadiendo variables, considerando interacciones o transformaciones de variables) puede resultar más engorroso; tendremos que andar preparando una nueva matriz para cada alternativa a explorar.

Afortunadamente, Statsmodels ha incluido desde hace algún tiempo la capacidad de definir un modelo mediante fórmulas, siguiendo la sintaxis que ya conoces del lenguaje R.


In [ ]:
# Cargamos la API con soporte a formulas
import statsmodels.formula.api as smf

# Definimos el modelo usando una formula como en R
model_f = smf.ols(formula = 'Sales ~ TV + Radio + Newspaper', data=ads)

# Ajustamos el modelo
mf_fitted = model_f.fit()

# Resumen
print(mf_fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Tue, 16 Jul 2019   Prob (F-statistic):           1.58e-96
Time:                        19:11:14   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9389      0.312      9.422      0.0

Como cabía esperar, los resultados del ajuste son los mismos. En realidad el modelo subyacente no cambia, sólo el mecanismo que hemos usado para definirlo.

Aparte de la sencillez de uso, tener la definición del modelo de forma textual aporta legibilidad. Es más fácil y rápido interpretar un modelo mediante su formulación que revisando la matriz de diseño.

> **Importante**. Si has revisado con atención el modelo y los resultados, tal vez ya te hayas dado cuenta de un cambio sutil al usar una fórmula. No hemos tenido que añadir un término constante explícito como ocurre con las matrices. La API para fórmulas lo hace por nosotros de forma automática (en línea con el funcionamiento en R).

#### Interacciones entre variables

Si el fenómeno que estamos modelando cumple que cuando todas las variables explicativas valen cero, entonces la respuesta es cero, podemos eliminar del ajuste el término constante o _intercept_. Eliminar un término se consigue simplemente restando en la fórmula. En nuestro ejemplo, la fórmula pasaría a ser

  `Sales ~ TV + Radio + Newspaper - 1`

Las interacciones entre variables se incluyen en las fórmulas igual que en R.

| Fórmula | Descripción |
|:--------|:------------|
| Y ~ X\*Z | Añadir la interacción entre las vars. X y Z. Equivalente a $Y = \beta_0 + \beta_1 X + \beta_2 Z + \beta_3 (X*Z)$ |
| Y ~ X:Z | Incluir solo la interacción entre las vars. X y Z. Equivalente a $Y = \beta_0 + \beta_3 (X*Z)$ |

#### Variables categóricas

Statsmodels también soporta el uso de variables categóricas en las fórmulas. La librería se encarga de hacer las transformaciones automáticas para añadir las variables _dummy_ necesarias por cada nivel.

Además, si tenemos una variable entera que queremos tratar como una variable categórica, basta con utilizar el operador `C()` con dicha variable en la fórmula.


In [ ]:
# Creamos una variable categórica 
# ¿el gasto en TV es bajo, medio o alto?
ads['TV_budget'] = pd.qcut(ads['TV'], [0, 0.15, 0.85, 1], labels=['LOW','MID','HIGH'])

# Creamos una variable entera
# 0 si hay más gasto en TV que en el resto
# 1 en caso contrario
ads['More_Radio_Or_News'] = ads.apply(lambda x: int(x['TV'] < x['Radio'] + x['Newspaper']), axis="columns")

# Definimos un modelo usando variables categóricas
model_c = smf.ols(formula = 'Sales ~ TV_budget + C(More_Radio_Or_News)', data=ads)

mc_fitted = model_c.fit()

print(mc_fitted.summary())


                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.438
Method:                 Least Squares   F-statistic:                     52.79
Date:                Tue, 16 Jul 2019   Prob (F-statistic):           4.69e-25
Time:                        19:11:45   Log-Likelihood:                -554.46
No. Observations:                 200   AIC:                             1117.
Df Residuals:                     196   BIC:                             1130.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

#### Transformaciones. Funciones en fórmulas.

Cuando observemos que existen relaciones no lineales entre variables, podemos embeber funciones para transformar las variables predictoras involucradas en nuestras fórmulas. El intérprete de fórmulas se encargará internamente de aplicar dichas funciones. Los únicos requisitos son que se trate de funciones vectorizadas, preparadas para operar sobre objetos tipo arrays o Series (como las funciones de NumPy); y además que las funciones sean accesibles en el contexto o espacio de nombres actual.

In [ ]:
# Definimos un modelo usando la raiz cuadrada del gasto en TV
# como predictor
model_fn = smf.ols(formula = 'Sales ~ np.sqrt(TV)', data=ads)

mfn_fitted = model_fn.fit()

print(mfn_fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     327.1
Date:                Tue, 16 Jul 2019   Prob (F-statistic):           8.39e-44
Time:                        19:11:51   Log-Likelihood:                -516.16
No. Observations:                 200   AIC:                             1036.
Df Residuals:                     198   BIC:                             1043.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.6724      0.667      4.004      